<a href="https://colab.research.google.com/github/gustavo-mota/ALC-Linear-Algebra/blob/main/ALC_Algoritmos_Fase02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center">
<h1>Curso de Algebra Linear</h1>
<h3>CKP8122</h3>
<h1>Programa de Mestrado e Doutorado em Ciência da Computação - MDCC</h1>
<h3>Universidade Federal do Ceará</h3>
<h2>Gustavo Mota</h2>
<h3>2022.2</h3>
</div>


In [ ]:
import numpy as np
import cmath
from operator import itemgetter
from scipy import linalg as scp
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import eigs, eigsh
from scipy.linalg import svd as sp_svd
#from math import sqrt

In [ ]:
sample_creto = np.array([
    [6, -12, -5, 16, -2],
    [-3, 6, 3, -9, 5],
    [-4, 8, 3, -10, 1]
])

sample_cretoB = np.array([-53, 29, 33])

Autovalores e Autovetores

In [ ]:
def potReg(A, x0, eps): # full simplificada
  x0 = np.ones(A.shape[0])
  x_ = x0/np.linalg.norm(x0)
  lbd_a = 0 #np.dot(x_.T, x)
  parar = False

  while(not parar):
    #x = np.matmul(A, x_) # na primeira iteração, já tenho o x_ do chute inicial
    x = A @ x_
    # lbd = np.dot(np.squeeze(x_), np.squeeze(x))
    lbd = x_ @ x
    if( abs( (lbd - lbd_a) /lbd )<eps ):
      parar = True
      x_ = x/np.linalg.norm(x)
    else:
      lbd_a = lbd
      x_ = x/np.linalg.norm(x)
  
  return (lbd, x_)

In [ ]:
def potReg4SVD(A, eps, x=None, max_it=100):
  n, m = A.shape
  x = np.ones(min(n, m)) if x is None else x.copy()
  parada = False
  egv = 0
  egv_ = egv

  b = A.T @ A if n > m else A @ A.T

  while(not parada):
      x_ = x / np.linalg.norm(x)
      x = b @ x_
      v = A @ x_ if n > m else A.T @ x_
      egv = np.linalg.norm(v)

      if abs((egv - egv_) / egv) < eps:
          parada = True
          break

      egv_ = egv

  x_ = x / np.linalg.norm(x)

  return (x_, egv)

In [ ]:
def potInv_v1(A, x0, eps):
  x0 = np.ones(A.shape[0])
  A_inv = np.linalg.pinv(A)
  lbd, x_ = potReg(A_inv, x0, eps)
  return (1/lbd, x_)

In [ ]:
def potInv_v2(A, x0, eps):
  x0 = np.ones(A.shape[0])
  x_ = x0/np.linalg.norm(x0)
  lbd_a = 0
  parada = False
  while(not parada):
    x = scp.solve(A, x_) # usar LU
    #lbd = np.dot(np.squeeze(x_), np.squeeze(x))
    lbd = x_ @ x
    if(abs((lbd - lbd_a)/lbd)<eps):
      parada = True
      x_ = x/np.linalg.norm(x, 2)
      print("parada inv2: ", x_)
    else:
      lbd_a = lbd
      x_ = x/np.linalg.norm(x, 2)
  # lbd = x_ @ x
  return (1/lbd, x_)

In [ ]:
def potShift(A, mi, x0, eps):
  A_ = A - mi*np.identity(A.shape[0])
  # parada = False
  lbd_, phi = potInv_v2(A_, x0, eps)
  lbd = lbd_ + mi

  '''while(not parada):
    lbd_, phi = potInv(A_, x0, eps)
    lbd = lbd_ + mi
    if(abs((lbd - lbd_a)/lbd)<eps):
      parada = True'''
  
  return lbd, phi

In [ ]:
sample_mat = np.array([ # padrão
    [4, 1, -2, 2],
    [1, 2, 0, 1],
    [-2, 0, 3, -2],
    [2, 1, -2, -1]
])

def matrizHouseholder(A_, idx, dim):
  # dim = n, j = idx
  # retorna Hj
  #N, v, v_ = np.zeros(dim)
  N, v, v_ = np.zeros((dim, 1)), np.zeros((dim, 1)), np.zeros((dim, 1)) # !
  print("matrizHouseholder", A_[:, idx], v)
  #v[idx+1:dim] = A_[:, idx]
  v[idx+1 : dim, 0] = A_[idx+1:dim, idx] # !
  print("v: ", v)
  L = np.linalg.norm(v, 2) # comprimento ou norma euclidiana
  v_[idx+1] = L
  print("L, v_: ", L, v_)
  N = v - v_
  n = N/np.linalg.norm(N)
  #h_ = np.identity(dim) - 2*(np.dot(np.squeeze(n), np.squeeze(n.T)))
  h_ = np.identity(dim) - 2*n@n.T # !
  return h_

def householder(A, dim):
  A_ = np.copy(A)
  H = np.identity(A.shape[0]) # EYE
  for idx in range(0, dim-2): # teste colocando 1 dado q inicia do 0
    h_ = matrizHouseholder(A_, idx, dim)
    #A_ = np.matmul(np.matmul(h_, A_), h_) # experimente alternar a ordem de prioridade das multiplicações 
    A_ = h_ @ A_ @ h_
    #H = np.matmul(H, h_)
    H = H @ h_
  return (A_, H) 

A_, H = householder(sample_mat, 4)
print(np.around(A_, 5))
#sample_mat.shape

matrizHouseholder [ 4  1 -2  2] [[0.]
 [0.]
 [0.]
 [0.]]
v:  [[ 0.]
 [ 1.]
 [-2.]
 [ 2.]]
L, v_:  3.0 [[0.]
 [3.]
 [0.]
 [0.]]
matrizHouseholder [ 3.          3.33333333 -1.33333333 -1.        ] [[0.]
 [0.]
 [0.]
 [0.]]
v:  [[ 0.        ]
 [ 0.        ]
 [-1.33333333]
 [-1.        ]]
L, v_:  1.6666666666666672 [[0.        ]
 [0.        ]
 [1.66666667]
 [0.        ]]
[[ 4.       3.      -0.      -0.     ]
 [ 3.       3.33333  1.66667 -0.     ]
 [-0.       1.66667 -1.32    -0.90667]
 [-0.       0.      -0.90667  1.98667]]


In [ ]:
sample_mqr = np.array([
    [12, -51, 4],
    [6, 167, -68],
    [-4, 24, -41]
])

def matHHmod(A_, idx, dim):
  v, v_ = np.zeros((dim, 1)), np.zeros((dim, 1))
  v[idx:dim, 0] = A_[idx:dim, idx]
  L = np.linalg.norm(v, 2) # norma euclidiana ou comprimento
  v_[idx] = L
  N = v - v_
  n = N/np.linalg.norm(N, 2)
  return np.identity(dim) - 2 * n @ n.T #np.identity(dim) - 2*(np.dot(np.squeeze(n), np.squeeze(n.T)))

def decompoQR(A, dim):
  A_ = np.copy(A)
  Q = np.identity(dim)
  for idx in range(0, dim-1):
    h_ = matHHmod(A_, idx, dim)
    A_ = np.matmul(h_, A_)
    Q = np.matmul(Q, h_)
  #print(Q)
  return (Q, A_)

def checker(D, dim):
  vecto = np.array([0])
  for idx in range(dim):
    #print("checker D[idx:, idx]", D[idx+1:dim, idx])
    vecto = np.concatenate((vecto, D[idx+1:dim, idx]))
    
  L = np.linalg.norm(vecto, 2)
  #print("checker L: ", L)
  return L

def metodoQR(T, H, dim, eps): # versão simétrica
  parada = False
  D = np.copy(T)
  # p = np.copy(H)
  p = np.eye(H.shape[0])
  while(not parada):
    Q, R = decompoQR(D, dim)
    D = np.matmul(R, Q)
    L = checker(D, dim) # pega os elementos abaixo da diagonal principal, ao quadrado e dps tira a raiz
    p = p @ Q
    if(L < eps):
      parada = True
  return D, p

def metodoQR4notSim(T, H, dim, eps): # versão não-simétrica
  parada = False
  D = np.copy(T)
  p = np.copy(H)
  old_L = 0.0 # np.zeros((dim, 1))
  while(not parada):
    Q, R = decompoQR(D, dim)
    D = np.matmul(R, Q)
    L = checker(D, dim) # pega os elementos abaixo da diagonal principal, ao quadrado e dps tira a raiz
    p = p @ Q
    if(abs(L - old_L) < eps):
      parada = True
    else:
      old_L = np.copy(L)
  return D, p

D, p = metodoQR(sample_mat, A_, 4, 0.001)
print(D)
lbd_reg, _ = potReg(sample_mat, [1], 0.0001)
lbd_inv1, _ = potInv_v1(sample_mat, [1], 0.0001)
lbd_inv2, _ = potInv_v2(sample_mat, [1], 0.0001)
lbd_shifted, _ = potShift(sample_mat, 5, [1], 0.0001)
print(lbd_reg, lbd_inv1, lbd_inv2, lbd_shifted)

[[ 6.84462111e+00 -1.00704982e-07  4.83989656e-08  8.17392064e-12]
 [-1.00704982e-07  2.26853119e+00  9.85503531e-04 -4.90171395e-16]
 [ 4.83989651e-08  9.85503531e-04 -2.19751676e+00  9.72250581e-16]
 [ 8.17356173e-12 -3.67002061e-20 -1.68404312e-20  1.08436446e+00]]
parada inv2:  [ 0.64240327 -0.54282273  0.52186827 -0.14253081]
parada inv2:  [ 0.71885438  0.22432631 -0.55501212  0.35339444]
6.844569183337946 1.0843910506767145 1.0843910506767143 6.844730865009231


In [ ]:
# q, r = np.linalg.qr(sample_mat, mode='reduced')
np.linalg.eigvals(sample_mat)

array([ 6.84462111, -2.19751698,  1.08436446,  2.26853141])

In [ ]:
# autovalor e autovetor
eigsh(sample_mat.astype(np.float), k=1, sigma=lbd_reg.astype(np.float))

<ipython-input-15-2b7e37406a45>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eigsh(sample_mat.astype(np.float), k=1, sigma=lbd_reg.astype(np.float))


(array([6.84462111]), array([[-0.71804596],
        [-0.22115299],
        [ 0.55735138],
        [-0.35335647]]))

In [ ]:
p

array([[ 0.71804596,  0.20167208,  0.17674968, -0.64226001],
       [ 0.221153  ,  0.78941059,  0.17827467,  0.54418785],
       [-0.55735136,  0.57969765, -0.28754012, -0.52022185],
       [ 0.35335648,  0.01048495, -0.92428262,  0.14398227]])

In [ ]:
'''
  [ 3 20 66]
  [ 7 78 56]
  [98 64 54]
'''

random_mat = np.random.rand(8,8) * 100
random_mat = random_mat.astype(np.int)
print(random_mat)

[[32 36 35 79 84 92 56 35]
 [82  8 86 32 43 13 82 28]
 [60 47 36 17  7 92 85 91]
 [26 94 69 51 53 57 50 30]
 [63 17 20 65 55 70 10 58]
 [68 66 64 76 20 47 78 77]
 [81 59  8 49 84 53 37 30]
 [94 76 33 10 41 49 51 53]]


<ipython-input-17-02def6a7b426>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_mat = random_mat.astype(np.int)


In [ ]:
random_mat = np.array([ # complexo
    [31, 29,  2, 40, 26, 17, 24, 54],
    [ 9, 76,  7,  2, 90,  4, 23, 13],
    [50, 49, 51, 19, 68,  5, 76, 89],
    [39, 85,  9, 72, 27, 72, 77, 57],
    [86, 38, 67, 49, 64, 35, 23, 81],
    [54, 71, 33, 97, 69, 82, 23, 18],
    [35, 14,  5, 22, 54, 12, 49, 67],
    [98, 32,  8, 82, 45, 75, 44,  5],
])

In [ ]:
np.linalg.eigvals(random_mat)

array([349.62298599 +0.j        ,  64.21325105+51.69594545j,
        64.21325105-51.69594545j, -55.68713131 +0.j        ,
         1.88709293+45.35508172j,   1.88709293-45.35508172j,
       -20.67967213 +0.j        ,  24.54312951 +0.j        ])

In [ ]:
A__, H_ = householder(random_mat, 8)
print(np.around(A_, 5))

matrizHouseholder [31  9 50 39 86 54 35 98] [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
v:  [[ 0.]
 [ 9.]
 [50.]
 [39.]
 [86.]
 [54.]
 [35.]
 [98.]]
L, v_:  158.88045820679145 [[  0.        ]
 [158.88045821]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]]
matrizHouseholder [ 70.53730916 263.59561066  69.10498782  66.19440289  25.84070634
  49.9131661   48.02480165 -39.39460583] [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
v:  [[  0.        ]
 [  0.        ]
 [ 69.10498782]
 [ 66.19440289]
 [ 25.84070634]
 [ 49.9131661 ]
 [ 48.02480165]
 [-39.39460583]]
L, v_:  127.17932658787039 [[  0.        ]
 [  0.        ]
 [127.17932659]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]]
matrizHouseholder [  7.07571019 116.02280785  23.98044212   9.62418171 -37.65187607
  33.7762635  -58.60545403  45.12815906] [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
v:  [[  0.        ]
 [  0.        ]
 [  0.        ]
 

In [ ]:
H_

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.05664636,  0.43425245,  0.16712385, -0.64504017,
        -0.07167782, -0.35184724, -0.48505114],
       [ 0.        ,  0.31470201,  0.39850025, -0.33863171, -0.41065783,
         0.09117721,  0.50257638,  0.44491865],
       [ 0.        ,  0.24546757,  0.40748515,  0.38573382,  0.43307137,
        -0.0551937 ,  0.51310032, -0.41357206],
       [ 0.        ,  0.54128746, -0.04598675, -0.3878045 ,  0.17613747,
        -0.68371345, -0.22111576, -0.08438071],
       [ 0.        ,  0.33987817,  0.23600732,  0.55769066,  0.13401748,
         0.03818995, -0.42236404,  0.56564473],
       [ 0.        ,  0.22029141,  0.27620846, -0.45537671,  0.33874344,
         0.63468017, -0.34400983, -0.17861645],
       [ 0.        ,  0.61681595, -0.59369425,  0.19957069, -0.25298283,
         0.33430702,  0.12133521, -0.19171082]])

In [ ]:
D, p = metodoQR4notSim(random_mat, A__, 8, 0.1)
print(D)

[[ 3.49622986e+02 -5.85335192e+00  9.81554859e+00  3.67616115e+01
   9.69423575e+00 -9.12219896e+01 -7.98399128e+01  7.89835445e-01]
 [ 5.73310053e-07  6.58725185e+01  4.20544541e+01 -1.34202448e+01
  -2.60511903e+01 -2.81845491e+00 -2.26418706e+00  3.17795338e+01]
 [ 5.38665465e-06 -6.36131892e+01  6.25540591e+01  7.21392699e+01
  -5.56742796e+00  6.62383913e+00  4.09755706e+01  2.37211414e+01]
 [-9.13262794e-09 -1.11927403e-04 -2.27657880e-04 -5.56847932e+01
  -2.46113513e+01 -3.29558006e+01  9.94322236e+00 -2.37862529e+01]
 [ 1.13955669e-09 -4.30619419e-07 -7.62031485e-08 -5.84611311e-03
  -9.97275545e+00  5.99651050e+01 -2.26932632e+01  8.59101899e+00]
 [-1.22440542e-09  3.28170904e-07  3.14406211e-07  1.32883955e-01
  -3.65768802e+01  1.37445280e+01  1.30248180e+01  3.56872976e+01]
 [ 1.89276382e-12 -9.59390418e-16  3.82248853e-17 -1.27835919e-10
   1.13714299e-07  1.55304174e-08  2.45456545e+01 -4.34408215e+00]
 [-1.01297920e-13  6.69029929e-18 -5.02504941e-18  3.67681212e-13
  -

In [ ]:
D_ = np.around(D, 5)
D_.astype(np.int64)

array([[349,  -5,   9,  36,   9, -91, -79,   0],
       [  0,  65,  42, -13, -26,  -2,  -2,  31],
       [  0, -63,  62,  72,  -5,   6,  40,  23],
       [  0,   0,   0, -55, -24, -32,   9, -23],
       [  0,   0,   0,   0,  -9,  59, -22,   8],
       [  0,   0,   0,   0, -36,  13,  13,  35],
       [  0,   0,   0,   0,   0,   0,  24,  -4],
       [  0,   0,   0,   0,   0,   0,   0, -20]])

In [ ]:
print("eigenvalues by numpy: ", np.linalg.eigvals(random_mat))
# print("eigenvalues by QR: ", D.diagonal())

eigenvalues by numpy:  [349.62298599 +0.j          64.21325105+51.69594545j
  64.21325105-51.69594545j -55.68713131 +0.j
   1.88709293+45.35508172j   1.88709293-45.35508172j
 -20.67967213 +0.j          24.54312951 +0.j        ]


In [ ]:
def unblock(d):
    bts = []
    idx = 0
    while idx <= d.shape[0]-1:
        if idx == d.shape[0]-1 or np.isclose(d[idx+1, idx], 0):
            bts.append(d[idx, idx])
            idx += 1
            continue

        bts.append(np.array([
            [d[idx, idx], d[idx, idx+1]],
            [d[idx+1, idx], d[idx+1, idx + 1]]
        ]))
        idx += 2
    return bts

ubk = unblock(D_)

def solve_block(block):
  a = 1
  b = block[0,0] + block[1, 1]
  c = block[0, 0]*block[1, 1] - block[0,1]*block[1, 0]
  delta = (b**2) - (4*a*c)

  s1 = (-b-cmath.sqrt(delta))/(2*a)
  s2 = (-b+cmath.sqrt(delta))/(2*a)

  return s1, s2

def eigenvalues4notSim(blocked):
  eigenvalues = []
  for block in blocked:
    #if type(block) != np.int64:
    if not isinstance(block, float):
      eigenvalues.append(solve_block(block))
      continue
    eigenvalues.append(block)
  return eigenvalues

eigenvalues4notSim(ubk)

[349.62299,
 ((-64.21329-51.69588643211953j), (-64.21329+51.69588643211953j)),
 ((9.969610577875446+0j), (55.68793942212456+0j)),
 13.74453,
 ((-24.543124736469764+0j), (20.679674736469764+0j))]

In [ ]:
for blk in ubk:
  print(type(blk)==np.int64)

False
False
False
False
False


In [ ]:
def checkerSimetria(matriz):
  b = matriz @ matriz.T
  if(np.allclose(matriz, b)):
    return 1
  return 0

def solver_eigenVV(matriz, dim, eps_QR=0.001, eps_QR2 = 0.1, shift_chute=5,
                   eps_potReg=0.0001, eps_potInv1=0.0001, eps_potInv2=0.0001, eps_potShift=0.0001
                   ):
  IsSim = checkerSimetria(matriz)

  A, H = householder(matriz, dim)
  print("Matriz de householder calculada! ", H)
  if(True):
    print("A matriz é simétrica!")
    D, p = metodoQR(matriz, A, dim, eps_QR)
    print("Decomposição QR calculada!")
    print("Matriz D: ", D)
    print("Matriz p:", p)
    eigenvalues = D_.diagonal()
  else:
    print('Matriz não simétrica!')
    D, p = metodoQR4notSim(matriz, A, dim, eps_QR2) # use 0.1
    print("Decomposição QR calculada!")
    print("Matriz D: ", D)
    print("Matriz p:", p)
    ubk = unblock(D_)
    print("Blocos encontrados! ", ubk)
    eigenvalues = eigenvalues4notSim(ubk)

  print("Eigenvalues obtidos: ", eigenvalues)
  print("Eigenvalues via numpy:",  np.linalg.eigvals(matriz).astype(np.int64))
  
  print("Testes via métodos de potência!")

  lbd_reg, _ = potReg(matriz, [1], eps_potReg)
  lbd_inv1, _ = potInv_v1(matriz, [1], eps_potInv1)
  lbd_inv2, _ = potInv_v2(matriz, [1], eps_potInv2)
  lbd_shifted, _ = potShift(matriz, shift_chute, [1], eps_potShift)
  print(
      "pot regular: ", lbd_reg, 
      "pot inversa 1: ", lbd_inv1, 
      "pot inversa 2: ", lbd_inv2, 
      "pot shifted: ", lbd_shifted
      )
  return {
      "sim": IsSim,
      "A": A,
      "HH": H,
      "Q": D,
      "R": p,
      # "blocks": ubk,
      "eVal": eigenvalues,
      "eVec": [],
      "pReg": lbd_reg, 
      "pInv1": lbd_inv1, 
      "pInv2": lbd_inv2, 
      "pShift": lbd_shifted
  }
# ouput = solver_eigenVV(random_mat, 8)

In [ ]:
eig01 = ouput['eVal'][0]
eigsh(random_mat.astype(np.float), k=1, sigma=eig01.astype(np.float))

<ipython-input-28-71074d7bdcb0>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eigsh(random_mat.astype(np.float), k=1, sigma=eig01.astype(np.float))


(array([349.62298587]), array([[-0.216277  ],
        [-0.22867636],
        [-0.34750042],
        [-0.43718863],
        [-0.43187483],
        [-0.48127187],
        [-0.22783908],
        [-0.34495188]]))

In [ ]:
output_creto = solver_eigenVV(prod_sample_creto, 3)
#prod_sample_creto.shape
#np.allclose(prod_sample_creto, prod_sample_creto @ prod_sample_creto.T)

matrizHouseholder [ 465. -251. -297.] [[0.]
 [0.]
 [0.]]
v:  [[   0.]
 [-251.]
 [-297.]]
L, v_:  388.85730030436616 [[  0.       ]
 [388.8573003]
 [  0.       ]]
Matriz de householder calculada!  [[ 1.          0.          0.        ]
 [ 0.         -0.645481   -0.76377633]
 [ 0.         -0.76377633  0.645481  ]]
A matriz é simétrica!
Decomposição QR calculada!
Matriz D:  [[ 7.90215445e+02 -6.35146674e-08 -2.19896641e-09]
 [-6.35145801e-08  7.40097624e-01 -1.48323570e-04]
 [-2.19903331e-09 -1.48323570e-04  4.44568937e-02]]
Matriz p: [[ 0.76708722  0.03644728  0.64050667]
 [-0.41413548  0.79063298  0.45098923]
 [-0.48996837 -0.61120462  0.62157857]]
Eigenvalues obtidos:  [349.62299  65.87252  62.55406 -55.68479  -9.97276  13.74453  24.54565
 -20.6822 ]
Eigenvalues via numpy: [790   0   0]
Testes via métodos de potência!
parada inv2:  [0.64051542 0.45117935 0.62143157]
parada inv2:  [ 0.05825253  0.80554206 -0.58966824]
pot regular:  790.2154454809591 pot inversa 1:  0.04445687069135247 p

In [ ]:
output_creto['eVal']

array([349.62299,  65.87252,  62.55406, -55.68479,  -9.97276,  13.74453,
        24.54565, -20.6822 ])

In [ ]:
aut, auv = potReg(prod_sample_creto, x0 = [1], eps=0.001)

In [ ]:
np.sqrt(aut)

28.11077098695372

In [ ]:
aut2, auv2 = potInv_v1(prod_sample_creto, x0 = [1], eps=0.001)

In [ ]:
def svd(A, k=None, epsilon=1e-10):
  n, m = A.shape
  A_ = A.copy()
  # print("n: ", n, " m: ", m)
      
  svd_ = []
  if k is None:
      k = min(n, m)

  for i in range(k):

      if n > m:
          # print("aq")
          v_, sgm = potReg4SVD(A_, 0.001)  # next singular vector
          u_unnormalized = A @ v
          # sigma = np.linalg.norm(u_unnormalized)  # next singular value
          u = u_unnormalized / sgm
          
      else:
          # print("here")
          u_, sgm = potReg4SVD(A_, eps=0.001)  # next singular vector
          v_unnormalized = np.dot(A.T, u_)
          # sigma = np.linalg.norm(v_unnormalized)  # next singular value
          v = v_unnormalized / sgm
      
      A_ -= sgm * np.outer(u_, v)
      svd_.append((sgm, u_, v))

  singular_values, us, vs = [np.array(x) for x in zip(*svd_)]
  return np.diag(singular_values), us.T, vs

sample_svd = np.array([
    [1,2,3],
    [4,5,6]
  ], dtype=np.float64)
sv, us, vs = svd(sample_svd) # sv, us, vs
sv
#sample_svd.shape

array([[9.508032  , 0.        ],
       [0.        , 0.77286964]])

In [ ]:
us @ sv @ vs

array([[1., 2., 3.],
       [4., 5., 6.]])

In [ ]:
sp_svd(sample_svd) # us, sv, vs

(array([[-0.3863177 , -0.92236578],
        [-0.92236578,  0.3863177 ]]),
 array([9.508032  , 0.77286964]),
 array([[-0.42866713, -0.56630692, -0.7039467 ],
        [ 0.80596391,  0.11238241, -0.58119908],
        [ 0.40824829, -0.81649658,  0.40824829]]))

In [ ]:
sample_creto = np.array([
    [6, -12, -5, 16, -2],
    [-3, 6, 3, -9, 1],
    [-4, 8, 3, -10, 1]
], dtype=float)

sample_cretoB = np.array([-53, 29, 33])

sv, us, vs = svd(sample_creto, k=None, epsilon=1e-10)

In [ ]:
prod_sample_creto = sample_creto @ sample_creto.T
prod_sample_creto02 = sample_creto.T @ sample_creto
prod_sample_creto02

array([[  61., -122.,  -51.,  163.,  -19.],
       [-122.,  244.,  102., -326.,   38.],
       [ -51.,  102.,   43., -137.,   16.],
       [ 163., -326., -137.,  437.,  -51.],
       [ -19.,   38.,   16.,  -51.,    6.]])

In [ ]:
aut1, auv1 = potReg(prod_sample_creto02, x0=[1], eps=0.0001)
aut2, auv2 = potInv_v1(prod_sample_creto02, x0=[1], eps=0.0001)
# aut3, auv3 = potShift(prod_sample_creto, mi=0.70, x0=[1], eps=0.0001)

In [ ]:
print('aut: ', np.sqrt(aut1), '\nauv: ', auv1)

aut:  28.110770986937954 
auv:  [ 0.27764458 -0.55528916 -0.23292665  0.74349788 -0.08673822]


In [ ]:
sv

array([[28.11077099,  0.        ,  0.        ],
       [ 0.        ,  0.86028809,  0.        ],
       [ 0.        ,  0.        ,  0.21084798]])

In [ ]:
us

array([[-0.76708722,  0.03637701,  0.64051067],
       [ 0.41413549,  0.79058349,  0.45107598],
       [ 0.48996837, -0.61127281,  0.6215115 ]])

In [ ]:
vs.T

array([[-0.27764458,  0.3389595 ,  0.01797526],
       [ 0.55528916, -0.677919  , -0.03595051],
       [ 0.23292665,  0.4138695 ,  0.0721331 ],
       [-0.74349788, -0.48877951, -0.12629094],
       [ 0.08673822,  0.12386161, -0.98855039]])

In [ ]:
us_ , sv_ , vs_ = sp_svd(sample_svd)

In [ ]:
# jacobi iterations
def jacob_iter(A, b, max_it, eps, ops=1):
  if ops == 1:
    x_ = np.ones((A.shape[0], 1))
  elif ops==2:
    x_ = np.ones((A.shape[0], 1))
    diago = A.diagonal()
    id = 0
    for el_a, el_b in zip(diago, b):
      print(el_a, el_b)
      x_[id] = el_b/el_a
      id += 1
  x_new = np.zeros((A.shape[0], 1))
  print("X_ : ", x_.T, x_new.shape)
  parada = False
  it = 0
  while (not parada):
    for idx, row in enumerate(A):
      # x_new[idx, 0] = ( b[idx] - sum( [el*x_[i] if i != idx else 0 for i, el in enumerate(row) ] ) )/row[idx]
      
      sum = 0
      for j, el in enumerate(row):
        if j != idx:
          print("sum: ", sum, "el: ", el, "x[j]: ", x_[j], "t: ", sum + el*x_[j], "j: ", j)
          sum += el*x_[j]
      
      x_new[idx, 0] = (b[idx] - sum)/row[idx] # testar com - em vez de  +
      print("row: ", row, "sum: ", sum, "x: ", x_new[idx, 0], "b: ", b[idx], "idx: ", idx)
    L_ = np.linalg.norm(x_, 2)
    L_new = np.linalg.norm(x_new, 2)
    if( abs(L_ - L_new) < eps ):
      parada = True
    print("x_: ", x_.T, " - x_new: ", x_new.T)
    print(abs(L_ - L_new))
    x_ = np.copy(x_new)
    it += 1
  return x_

sample_wen = np.array([
    [2, -1, 0],
    [-1, 2, 1],
    [0, -1, 2]
], dtype=np.float64)

sample_bWen = np.array([
    [0, 1, 2]
], dtype=np.float64).T

out = jacob_iter(sample_wen, sample_bWen, 5000, 0.01, ops=2)
print("output", out.T) # 0.625, 1.5, 1.625

2.0 [0.]
2.0 [1.]
2.0 [2.]
X_ :  [[0.  0.5 1. ]] (3, 1)
sum:  0 el:  -1.0 x[j]:  [0.5] t:  [-0.5] j:  1
sum:  [-0.5] el:  0.0 x[j]:  [1.] t:  [-0.5] j:  2
row:  [ 2. -1.  0.] sum:  [-0.5] x:  0.25 b:  [0.] idx:  0
sum:  0 el:  -1.0 x[j]:  [0.] t:  [0.] j:  0
sum:  [0.] el:  1.0 x[j]:  [1.] t:  [1.] j:  2
row:  [-1.  2.  1.] sum:  [1.] x:  0.0 b:  [1.] idx:  1
sum:  0 el:  0.0 x[j]:  [0.] t:  [0.] j:  0
sum:  [0.] el:  -1.0 x[j]:  [0.5] t:  [-0.5] j:  1
row:  [ 0. -1.  2.] sum:  [-0.5] x:  1.25 b:  [2.] idx:  2
x_:  [[0.  0.5 1. ]]  - x_new:  [[0.25 0.   1.25]]
0.15672088964830144
sum:  0 el:  -1.0 x[j]:  [0.] t:  [0.] j:  1
sum:  [0.] el:  0.0 x[j]:  [1.25] t:  [0.] j:  2
row:  [ 2. -1.  0.] sum:  [0.] x:  0.0 b:  [0.] idx:  0
sum:  0 el:  -1.0 x[j]:  [0.25] t:  [-0.25] j:  0
sum:  [-0.25] el:  1.0 x[j]:  [1.25] t:  [1.] j:  2
row:  [-1.  2.  1.] sum:  [1.] x:  0.0 b:  [1.] idx:  1
sum:  0 el:  0.0 x[j]:  [0.25] t:  [0.] j:  0
sum:  [0.] el:  -1.0 x[j]:  [0.] t:  [0.] j:  1
row:  [ 0. 

In [ ]:
sample_jacobi = np.array([
    [4, -1, -1],
    [-2, 6, 1],
    [-1, 1, 7]
])

sample_bJaco = np.ones((3, 1))
sample_bJaco[0, 0] = 3
sample_bJaco[1, 0] = 9
sample_bJaco[2, 0] = -6

# https://www.quantstart.com/articles/Jacobi-Method-in-Python-and-NumPy/
A = np.array([[2.0,1.0],[5.0,7.0]])
b = np.array([11.0,13.0])
# saída esperada: 7, -3

# out = jacob_iter(sample_jacobi, sample_bJaco, 5, 0.01, ops=2)
# print(out)

out = jacob_iter(A, b, 5000, 0.01)
print(out)

# out = jacob_iter(sample_wen, sample_bWen, 5000, 0.01, ops=2)
# print(out)

X_ :  [[1. 1.]] (2, 1)
sum:  0 el:  1.0 x[j]:  [1.] t:  [1.] j:  1
row:  [2. 1.] sum:  [1.] x:  5.0 b:  11.0 idx:  0
sum:  0 el:  5.0 x[j]:  [1.] t:  [5.] j:  0
row:  [5. 7.] sum:  [5.] x:  1.1428571428571428 b:  13.0 idx:  1
x_:  [[1. 1.]]  - x_new:  [[5.         1.14285714]]
3.714735886488689
sum:  0 el:  1.0 x[j]:  [1.14285714] t:  [1.14285714] j:  1
row:  [2. 1.] sum:  [1.14285714] x:  4.928571428571429 b:  11.0 idx:  0
sum:  0 el:  5.0 x[j]:  [5.] t:  [25.] j:  0
row:  [5. 7.] sum:  [25.] x:  -1.7142857142857142 b:  13.0 idx:  1
x_:  [[5.         1.14285714]]  - x_new:  [[ 4.92857143 -1.71428571]]
0.08924869194646856
sum:  0 el:  1.0 x[j]:  [-1.71428571] t:  [-1.71428571] j:  1
row:  [2. 1.] sum:  [-1.71428571] x:  6.357142857142857 b:  11.0 idx:  0
sum:  0 el:  5.0 x[j]:  [4.92857143] t:  [24.64285714] j:  0
row:  [5. 7.] sum:  [24.64285714] x:  -1.6632653061224494 b:  13.0 idx:  1
x_:  [[ 4.92857143 -1.71428571]]  - x_new:  [[ 6.35714286 -1.66326531]]
1.352929371626204
sum:  0 e

In [ ]:
# gauss-seidel iterations

def gs_iter(A, b, eps, sor=1):
  parada = False
  x_ = np.ones((A.shape[0], 1)).astype(np.float64)
  diago = A.diagonal()
  id = 0
  for el_a, el_b in zip(diago, b):
    print(el_a, el_b)
    x_[id] = el_b/el_a
    id += 1

  x_new = np.zeros((A.shape[0], 1))
  itr = 0
  while(not parada):
    #print("iteração: ", itr)
    itr += 1
    for i in range(A.shape[0]):
      #print('i', i)
      sum1 = sum([
          A[i, j]*x_[j] for j in range(i)
      ])

      sum2 = sum([
          A[i, j]*x_[j] for j in range(i+1, A.shape[0])
      ])

      # gaus-seidel : sor = 1 ; sor : sor > 1
      x_new[i] = (1 - sor)*x_[i] + (sor/A[i,i]) * ( b[i, 0]-sum1-sum2) 
      # print(x_new[i])
    
    L_ = np.linalg.norm(x_, 2)
    L_new = np.linalg.norm(x_new, 2)
    #print("sum1: ", sum1, "sum2: ", sum2)
    if( abs(L_ - L_new) < eps ):
      parada = True
    x_ = np.copy(x_new)
  
  return x_new

#gs_iter(sample_wen, sample_bWen, 0.001) # 0.5625 1.5625 1.7813
A = np.array([[2.0,1.0],[5.0,7.0]])
b = np.array([11.0,13.0])
gs_iter(A.astype(np.float64), np.reshape(b, (2, 1)).astype(np.float64), 0.00001, sor=1) 

2.0 [11.]
7.0 [13.]


array([[ 7.11108049],
       [-3.22220835]])

In [ ]:
scp.solve(A, b)

array([ 7.11111111, -3.22222222])

In [ ]:
def SOR(A, b, omg, init_g, eps):
  phi = init_g[:]
  res = np.linalg.norm(np.matmul(A, phi) - b)
  while(res > eps):
    for i in range(A.shape[0]):
      sigma = 0
      for j in range(A.shape[1]):
        if(j != i):
          sigma += A[i][j] * phi[j]
      phi[i] = (1 - omg) * phi[i] + (omg / A[i][i]) * (b[i] - sigma)
    res = np.linalg.norm(np.matmul(A, phi) - b)
    # print('Residual: {0:10.6g}'.format(res))
  return phi

SOR(A, b, 1.5, np.zeros(2), 0.0001)

array([ 7.11112034, -3.22223376])

In [ ]:
def grad_conj(A, b, x0=None, eps=1e-5, maxiter=100):
    n = b.shape[0]
    if not x0:
        x0 = np.zeros(n)
        print(x0)
    grad0 = np.dot(A, x0) - b        
    print("grad0: ", grad0)
    d = - grad0                      
    
    for i in range(maxiter):
        alpha = np.dot(grad0.T, grad0) / np.dot(np.dot(d.T, A), d)
        print(" alpha: ", alpha)
        # x0 = x0 + d*(alpha[0])
        x0 = x0 + d*(alpha)
        # gradi = grad0 + np.dot(A*(alpha[0]), d)
        gradi = grad0 + np.dot(A*(alpha), d)
        # print(i, np.linalg.norm(gradi))

        if np.linalg.norm(gradi) < eps:
            # print("alpha: ", alpha, " x0", x0.shape, " gradi: ", gradi, " grad0: ", grad0, "betha: ", betai, " d: ", d.shape)
            return x0
        betai = np.dot(gradi.T, gradi) / np.dot(grad0.T, grad0)
        # d = - gradi + (betai[0])*d
        d = - gradi + (betai)*d
        grad0 = gradi
    # print("alpha: ", alpha, " x0", x0.shape, " gradi: ", gradi, " grad0: ", grad0, "betha: ", betai, " d: ", d.shape)
    return x0

sample_gc = np.array([
    [ 2.5409, -0.0113],
    [-0.0113,  0.5287]
  ])
# sample_gcB = np.array([
#     [1.3864,  0.3719]
# ]).T
sample_gcB = np.array( [1.3864,  0.3719] )
grad_conj(sample_gc, sample_gcB)

[0. 0.]
grad0:  [-1.3864 -0.3719]
 alpha:  0.4166369074833467
 alpha:  1.7868440888573531


array([0.5488139 , 0.71515339])

In [ ]:
cg(sample_gc, sample_gcB)

(array([0.5488139 , 0.71515339]), 0)